# Chart Descriptions

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/andrewm4894/netdata-gpt-notebooks/blob/main/notebooks/chart_descriptions/chart_descriptions.ipynb)

In [121]:
# if running in colab uncomment the following line and run it to install the required packages
#!pip install python-dotenv netdata-pandas openai

In [122]:
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from netdata_pandas.data_cloud import get_data_cloud
import openai
import pprint as pp
from urllib.parse import urlparse
import requests
import json

# load tokens from .env file
load_dotenv()

NETDATA_API_TOKEN = os.getenv('NETDATA_API_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

openai.api_key = OPENAI_API_KEY

In [123]:
# inputs
space_id = 'ea93d7b8-0df6-45c0-b13d-1560996c89eb' 
room_id = 'd8a4e0c5-7c79-4145-900e-83a9f06fcb6a'
chart = None

In [124]:
def get_charts_cloud(space_id, room_id, api_token=None, base_url='https://app.netdata.cloud', node_ids=[]):
    """Get charts from netdata cloud api.
    """
    
    if api_token is None:
        api_token = os.getenv('NETDATA_API_TOKEN')
    
    base_url = 'https://app.netdata.cloud'
    url = f'{base_url}/api/v2/spaces/{space_id}/rooms/{room_id}/charts'
    headers = {'Accept': '*/*', 'Content-Type': 'application/json', 'Authorization': f'Bearer {api_token}'}
    data = {
        'filter': {
            'nodeIDs': node_ids,
        }
    }
    r = requests.post(url, headers=headers, data=json.dumps(data))
    
    if r.status_code != 200:
        
        print(f'Error: {r.status_code, r.text}')
        return None
    
    else:
            
        return r.json()['results']


charts = get_charts_cloud(space_id, room_id, api_token=NETDATA_API_TOKEN)

In [125]:
if chart == None:
    chart = np.random.choice(list(charts.keys()))

print(chart)

windows.processes_cpu_utilization


In [126]:
df = get_data_cloud(space_id, room_id, chart)
print(df.shape)
print(df.head())

dimensions = df.columns
print(dimensions)

(13, 11)
                      LogonUI   MsMpEng  System  WmiPrvSE  conhost  csrss  \
time                                                                        
2023-04-01 20:51:57  0.000000  0.151436     0.0  0.320015      0.0    0.0   
2023-04-01 20:52:00  0.000000  0.000000     0.0  0.320176      0.0    0.0   
2023-04-01 20:52:03  0.000000  0.000000     0.0  0.249129      0.0    0.0   
2023-04-01 20:52:06  0.022827  0.000000     0.0  0.183554      0.0    0.0   
2023-04-01 20:52:09  0.091310  0.000000     0.0  0.128924      0.0    0.0   

                        lsass  services  ssm-agent-worker   svchost  \
time                                                                  
2023-04-01 20:51:57  0.151436  0.181467          0.174290  1.886459   
2023-04-01 20:52:00  0.000000  0.114269          0.114269  3.027583   
2023-04-01 20:52:03  0.000000  0.230230          0.157146  3.042908   
2023-04-01 20:52:06  0.000000  0.285668          0.148585  2.651562   
2023-04-01 20:52:09  0.00

In [127]:
chart_json = {}
chart_json['id'] = chart
chart_json['title'] = charts[chart]['title']
chart_json['dimensions'] = list(dimensions)
chart_json['units'] = charts[chart]['units']
chart_json['family'] = charts[chart]['family']
chart_json['context'] = charts[chart]['context']
chart_json['chart_type'] = charts[chart]['chartType']
chart_json['chart_labels'] = charts[chart]['chartLabels']
chart_json

{'id': 'windows.processes_cpu_utilization',
 'title': 'CPU usage (100% = 1 core)',
 'dimensions': ['LogonUI',
  'MsMpEng',
  'System',
  'WmiPrvSE',
  'conhost',
  'csrss',
  'lsass',
  'services',
  'ssm-agent-worker',
  'svchost',
  'windows_exporter'],
 'units': 'percentage',
 'family': 'processes',
 'context': 'windows.processes_cpu_utilization',
 'chart_type': 'stacked',
 'chart_labels': None}

In [128]:
prompt = f"""
You are an experienced SRE and sysadmin.

You are monitoring your infrastructure using Netdata Cloud.

You are documenting individual charts and their dimensions to help other users.

THe chart_json object is available to you and follows the format below:

```json
{{
    "id": "the chart id",
    "title": "the chart title",
    "dimensions": "a list of the dimensions",
    "units": "the units of the chart",
    "family": "the menu family of the chart",
    "context": "context of the chat. it follows structure like <type>.<name> where <type> also impacts where in the menu the chart appears and so can give hints of what it might relate to",
    "chart_type": "usually line or stacked or area - this is the type of chart"
}}
```

Here is the individual chart_json object describing the chart you are documenting:

```json
{chart_json}
```

Can you write short, useful and educational description of the chart and its dimensions? 

Please follow format like this:

```json
{{
    "chart_id": "{chart}",
    "chart_description": "<add description here>",
    "dimension_descriptions": [
        {{
            "<dimension name>": "<add dimension description here>",
            ...
        }}
}}
```
"""
print(prompt)


You are an experienced SRE and sysadmin.

You are monitoring your infrastructure using Netdata Cloud.

You are documenting individual charts and their dimensions to help other users.

THe chart_json object is available to you and follows the format below:

```json
{
    "id": "the chart id",
    "title": "the chart title",
    "dimensions": "a list of the dimensions",
    "units": "the units of the chart",
    "family": "the menu family of the chart",
    "context": "context of the chat. it follows structure like <type>.<name> where <type> also impacts where in the menu the chart appears and so can give hints of what it might relate to",
    "chart_type": "usually line or stacked or area - this is the type of chart"
}
```

Here is the individual chart_json object describing the chart you are documenting:

```json
{'id': 'windows.processes_cpu_utilization', 'title': 'CPU usage (100% = 1 core)', 'dimensions': ['LogonUI', 'MsMpEng', 'System', 'WmiPrvSE', 'conhost', 'csrss', 'lsass', 'ser

In [129]:
# build messages list to pass to openai
messages=[
    {"role": "user", "content": prompt}
]

In [130]:
# call openai api
completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
)

In [131]:
reply_content = completion.choices[0].message.content
#pp.pprint(reply_content)
print(reply_content)

{
    "chart_id": "windows.processes_cpu_utilization",
    "chart_description": "This chart displays the CPU usage of processes running on a Windows system. The chart is stacked, allowing you to see the CPU usage of individual processes as well as the total CPU usage.",
    "dimension_descriptions": [
        {
            "LogonUI": "The process that handles user authentication and login on Windows systems.",
            "MsMpEng": "Microsoft Security Essentials antivirus software process.",
            "System": "The central operating system kernel process.",
            "WmiPrvSE": "Windows Management Instrumentation Provider Service used by various system components.",
            "conhost": "Console Host process, responsible for handling command prompt windows.",
            "csrss": "Client/Server Runtime Subsystem process that manages system windows and console windows.",
            "lsass": "Local Security Authority Subsystem Service process that manages security policies and 